In [1]:
!nvidia-smi 

Wed Jul 23 07:25:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.61                 Driver Version: 572.61         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8              6W /   46W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Document Preprocessing and Embedding

In [2]:
import os 
import requests
pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
    print(f"[Info] File doesn't exist , downloading ...")
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
    file_name = pdf_path
    response = requests.get(url)
    if response.status_code == 200 : 
        with open(file_name , "wb") as file  : 
            file.write(response.content)
        print(f"[INFO] the file has been downloaded and saved as {file_name}")
    else : 
        print(f"[INFO Failed to download the file . Status Code : {response.status_code} ]")
else : 
    print(f"[INFO] file already exists")

[INFO] file already exists


In [3]:
import fitz
from tqdm import tqdm

def text_formatter(text:str) -> str : 
    cleaned_text = text.replace("\n"," ").strip()
    return cleaned_text
def open_read_pdf(pdf_path:str)->list[dict] : 
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number , page in tqdm(enumerate(doc)) : 
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({"page_number" : page_number, 
                                "page_char_count" : len(text),                ## for exploratory data analysis
                                "page_word_count" : len(text.split(" ")),     ## for exploratory data analysis
                                "page_sentence_count" : len(text.split(".")), ## for exploratory data analysis
                                "page_token_count" : len(text)/4, ## every token consists of 4 chars              
                                "text":text})
        
    return pages_and_texts
pages_and_texts = open_read_pdf(pdf_path)
pages_and_texts[50]

1208it [00:01, 1096.16it/s]


{'page_number': 50,
 'page_char_count': 1320,
 'page_word_count': 215,
 'page_sentence_count': 4,
 'page_token_count': 330.0,
 'text': 'Minerals  Major Functions  Macro  Sodium  Fluid balance, nerve transmission, muscle contraction  Chloride  Fluid balance, stomach acid production  Potassium  Fluid balance, nerve transmission, muscle contraction  Calcium  Bone and teeth health maintenance, nerve transmission,  muscle contraction, blood clotting  Phosphorus  Bone and teeth health maintenance, acid-base balance  Magnesium  Protein production, nerve transmission, muscle  contraction  Sulfur  Protein production  Trace  Iron  Carries oxygen, assists in energy production  Zinc  Protein and DNA production, wound healing, growth,  immune system function  Iodine  Thyroid hormone production, growth, metabolism  Selenium  Antioxidant  Copper  Coenzyme, iron metabolism  Manganese  Coenzyme  Fluoride  Bone and teeth health maintenance, tooth decay  prevention  Chromium  Assists insulin in glucose m

In [4]:
import pandas as pd 
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,0,29,4,1,7.25,Human Nutrition: 2020 Edition
1,1,0,1,1,0.00,
2,2,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,3,212,32,3,53.00,Human Nutrition: 2020 Edition by University of...
4,4,797,145,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [5]:
df.describe().round()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,1208.0,1208.0,1208.0,1208.0,1208.0
mean,604.0,1148.0,198.0,14.0,287.0
std,349.0,560.0,96.0,10.0,140.0
min,0.0,0.0,1.0,1.0,0.0
25%,302.0,762.0,134.0,8.0,190.0
50%,604.0,1232.0,214.0,13.0,308.0
75%,905.0,1604.0,271.0,19.0,401.0
max,1207.0,2308.0,429.0,82.0,577.0
